In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import asyncio
import datetime
from datetime import datetime
from datetime import timedelta, timezone
from typing import Optional
from dotenv import load_dotenv
from operator import itemgetter

load_dotenv()
PWD = os.getenv("PWD")
db_name = PWD + "\\database" + "\\RVNUSDT.db"
import sys

sys.path.insert(1, PWD + "\\modules")
from alg_modules.alg_handler import AlgHandler
from plot_modules.candle_plot import CandlePlot
from collections import deque
from paper_trade import PaperTrader
import time
import logging

DEBUG = __debug__
LOG_FILE_NAME = "log_file_name.log"
format = "%(asctime)s [%(levelname)s]: %(message)s"
logger = logging.basicConfig(
    filename=LOG_FILE_NAME if not DEBUG else None,
    format=format,
    encoding="utf-8",
    level=logging.INFO,
)
if not DEBUG:
    logging.getLogger(logger).addHandler(logging.StreamHandler())

from stop_loss import StopLoss
from trade_strategy import TradeStrategy
from wss_thread import WssThread
from api_modules.open_binance_api import OpenBinanceApi
import pytz
tzdata = pytz.timezone('Europe/Moscow') 

In [2]:
def compute_timedelta(dt: datetime):
    if dt.tzinfo is None:
        dt = dt.astimezone()
    now = datetime.now(timezone.utc)
    return max((dt - now).total_seconds(), 0)
async def sleep_until(when: datetime, result = None):
    """|coro|
    Sleep until a specified time.
    If the time supplied is in the past this function will yield instantly.
    .. versionadded:: 1.3
    Parameters
    -----------
    when: :class:`datetime.datetime`
        The timestamp in which to sleep until. If the datetime is naive then
        it is assumed to be local time.
    result: Any
        If provided is returned to the caller when the coroutine completes.
    """
    delta = compute_timedelta(when)
    return await asyncio.sleep(delta, result)
def get_data():
    df = OpenBinanceApi.get_df(
        pair = 'RVNUSDT',
        interval = '1m',
        limit = 1,
    )
    return df

    

In [3]:
server_time = datetime.fromtimestamp(OpenBinanceApi.server_time()/1000)
my_time = datetime.now()
delay = server_time - my_time
display(f'server_time: {server_time}')
display(f'my_time: {my_time}')
delay.total_seconds()

'server_time: 2022-02-22 09:59:26.571000'

'my_time: 2022-02-22 09:59:19.422188'

7.148812

In [4]:
# w = WssThread(
#     url='wss://stream.binance.com:9443/ws/rvnusdt@ticker',
#     maxlen=10,
#     )
# w.start()

# STOP_LOSS_ENABLED=True, 
# STOP_LOSS_THRESHOLD=-1.3, 

# # df = OpenBinanceApi.get_df(
# #     pair = 'RVNUSDT',
# #     interval = '5m',
# #     limit = 1000,
# #     ) 
# # stop_loss_trade_flag = False
# # MA_list = (2, 25, 100, 200)

# # window = deque(maxlen=200)
# # p_trdr = PaperTrader(
# #     main_currency_label='RVN',
# #     secondary_currency_label='USD',
# #     main_currency_amount=100,
# #     secondary_currency_amount=0,
# #     fee=0.1,
# # )
# # trade_data = pd.DataFrame(
# #     columns = p_trdr.get_df(timestamp=df.iloc[-1]['date_created']).columns.values
# # )
# # stop_loss = StopLoss(
# #     STOP_LOSS_THRESHOLD=STOP_LOSS_THRESHOLD,
# # )
# # # init alg
# # alg = AlgHandler(
# #     df=pd.DataFrame([]),
# #     MA_list=MA_list,
# #     )
# # dt = datetime.datetime.today()
# try:
#     while True:
#         # get_df
#         df = OpenBinanceApi.get_df(
#                 pair = 'RVNUSDT',
#                 interval = '5m',
#                 limit = 2,
#             ) 
#         # wait for new data
#         dt = tzdata.localize(
#                 datetime\
#                     .datetime\
#                     .fromtimestamp(
#                         int(df.Real_Date[-1:])/1000
#                     )
#             )
#         desync = datetime.datetime.now() - dt
#         wait_for = datetime.timedelta(seconds=5*60) - desync
#         logging.info(f"sleep for: {wait_for}")
#         await async.sleep(wait_for)

#         # logging.info(f"{dt}")
#         # display(df)
        
#         # if w.get_data() is not None:
#         #     print(itemgetter('a', 'b', 'c')(w.get_data()))
#         # new_dt = dt + datetime.timedelta(minutes=5)
#         # logging.info(f"sleeping till: {new_dt}")
#         # await sleep_until(new_dt)

#         #region
#         # for _, row in df.iterrows():
#         # if 1: # wait for new data comes in
#         #     window.append(dict(row))
#         #     df_ = pd.DataFrame(window)
#         #     alg.update_data(df_)
#         #     alg.calculate(val_col='open_', time_col='open_time',)
#         #     do_trade, cross_type = alg.evaluate()
#         #     if STOP_LOSS_ENABLED:
#         #         stop_loss_trade_flag, trade_data = stop_loss.stop_loss_alg(
#         #                                                 trade_data=trade_data,
#         #                                                 p_trdr=p_trdr,
#         #                                                 row=row,
#         #                                             )

#         #     if do_trade:
#         #         trade_data, stop_loss_trade_flag = TradeStrategy.trade_alg(stop_loss_trade_flag, trade_data, p_trdr, row, cross_type)
#         # if w.get_data() is not None:
#         #     print(itemgetter('a', 'b', 'c')(w.get_data()))

#         # time.sleep(1)
#         #endregion
# finally:
#     # close thread
#     w.close()
    

In [6]:
DATA_AWAIT_TIME = 1 # seconds
SERVER_DELAY = 0 # seconds
time_mark = datetime.now() + timedelta(hours=3)
server_delay_list = []
logging.info(f'==={delay.total_seconds()=}')
while time_mark > datetime.now():
# while 1:
    # pepe
    logging.info('===get new===')
    df = get_data()
    dt = datetime.fromtimestamp(int(df.Real_Date[-1:])/1000)

    server_time = datetime.fromtimestamp(OpenBinanceApi.server_time()/1000)
    local_time = datetime.now() + delay
    logging.info(f'server time: {server_time}   {server_time.minute=}, {dt.minute=}')
    logging.info(f'local time: {local_time}   {local_time.minute=}, {dt.minute=}')
    dt_ = dt - delay + timedelta(seconds=SERVER_DELAY) + timedelta(seconds=60)
    
    if server_time.minute == dt.minute: # solve 0 vs 59 problem
        logging.info('+++===sucksess===+++')
        server_valid_time = server_time
        logging.info(f'server valid time: {server_valid_time}')
        server_delay = dt - server_valid_time
        logging.info(f'server delay: {server_delay.total_seconds()}')
        server_delay_list.append(server_delay)
        logging.info(f'sleep till: {dt_}')
        await sleep_until(dt_)
    else:
        logging.info('---not valid---')
        # sleep_till = dt + timedelta(seconds=60)
        # logging.info(f'sleep till: {sleep_till}')
        logging.info('sleep 1 sec')
        await asyncio.sleep(DATA_AWAIT_TIME)
server_delay_list


2022-02-22 10:07:48,916 [INFO]: ===delay.total_seconds()=7.148812
2022-02-22 10:07:48,919 [INFO]: ===get new===
2022-02-22 10:07:50,192 [INFO]: server time: 2022-02-22 10:07:57.336000   server_time.minute=7, dt.minute=7
2022-02-22 10:07:50,194 [INFO]: local time: 2022-02-22 10:07:57.341076   local_time.minute=7, dt.minute=7
2022-02-22 10:07:50,195 [INFO]: +++===sucksess===+++
2022-02-22 10:07:50,196 [INFO]: server valid time: 2022-02-22 10:07:57.336000
2022-02-22 10:07:50,197 [INFO]: server delay: -57.336
2022-02-22 10:07:50,198 [INFO]: sleep till: 2022-02-22 10:07:52.851188
2022-02-22 10:07:52,852 [INFO]: ===get new===
2022-02-22 10:07:53,615 [INFO]: server time: 2022-02-22 10:08:00.755000   server_time.minute=8, dt.minute=7
2022-02-22 10:07:53,616 [INFO]: local time: 2022-02-22 10:08:00.763824   local_time.minute=8, dt.minute=7
2022-02-22 10:07:53,617 [INFO]: ---not valid---
2022-02-22 10:07:53,618 [INFO]: sleep 1 sec
2022-02-22 10:07:54,632 [INFO]: ===get new===
2022-02-22 10:07:56,

[datetime.timedelta(days=-1, seconds=86342, microseconds=664000),
 datetime.timedelta(days=-1, seconds=86394, microseconds=682000),
 datetime.timedelta(days=-1, seconds=86396, microseconds=963000),
 datetime.timedelta(days=-1, seconds=86393, microseconds=244000),
 datetime.timedelta(days=-1, seconds=86389, microseconds=571000),
 datetime.timedelta(days=-1, seconds=86392, microseconds=866000),
 datetime.timedelta(days=-1, seconds=86395, microseconds=74000),
 datetime.timedelta(days=-1, seconds=86393, microseconds=427000),
 datetime.timedelta(days=-1, seconds=86396, microseconds=74000),
 datetime.timedelta(days=-1, seconds=86389, microseconds=673000),
 datetime.timedelta(days=-1, seconds=86394, microseconds=707000),
 datetime.timedelta(days=-1, seconds=86395, microseconds=503000),
 datetime.timedelta(days=-1, seconds=86396, microseconds=498000),
 datetime.timedelta(days=-1, seconds=86394, microseconds=587000),
 datetime.timedelta(days=-1, seconds=86388, microseconds=884000),
 datetime.ti

In [11]:
server_delay_list[0].total_seconds()
sdl = [i.total_seconds() for i in server_delay_list]
sdl_f = [i for i in sdl if i > -60]
sdl_f.pop(0)


-57.336

In [12]:
len(sdl_f)

180

In [29]:
sdl_f

[-5.318,
 -3.037,
 -6.756,
 -10.429,
 -7.134,
 -4.926,
 -6.573,
 -3.926,
 -10.327,
 -5.293,
 -4.497,
 -3.502,
 -5.413,
 -11.116,
 -11.566,
 -12.111,
 -11.387,
 -2.606,
 -6.152,
 -3.093,
 -2.624,
 -2.535,
 -10.566,
 -10.158,
 -4.317,
 -6.122,
 -8.004,
 -6.253,
 -10.182,
 -8.0,
 -2.535,
 -6.081,
 -2.802,
 -9.635,
 -9.655,
 -8.396,
 -5.326,
 -9.048,
 -9.328,
 -9.654,
 -9.777,
 -8.496,
 -6.896,
 -9.758,
 -5.29,
 -2.498,
 -3.039,
 -5.503,
 -3.126,
 -9.63,
 -9.123,
 -2.567,
 -7.802,
 -2.603,
 -2.704,
 -5.284,
 -4.969,
 -4.624,
 -3.533,
 -23.465,
 -10.783,
 -10.072,
 -10.261,
 -5.346,
 -9.836,
 -11.157,
 -3.526,
 -10.32,
 -8.92,
 -8.588,
 -9.296,
 -1.254,
 -10.529,
 -3.031,
 -3.535,
 -7.566,
 -4.825,
 -5.075,
 -10.969,
 -9.388,
 -6.651,
 -4.336,
 -10.669,
 -10.969,
 -6.71,
 -6.321,
 -4.551,
 -10.488,
 -10.352,
 -5.039,
 -8.621,
 -8.537,
 -3.819,
 -6.778,
 -10.609,
 -9.789,
 -10.823,
 -9.206,
 -9.83,
 -6.135,
 -10.806,
 -4.918,
 -2.642,
 -7.979,
 -8.871,
 -2.687,
 -10.088,
 -9.873,
 -10.25,
 -

In [13]:
import plotly.express as px
# server_delay_list.pop(0)
fig = px.line(sdl_f)

fig

In [50]:
fig = px.histogram(sdl_f, 
    nbins=180,
    marginal="violin", # or violin, rug, box
    )
fig.update_layout(
    annotations=[dict(
                x=-9.6, 
                y=0.8, 
                xref='x', 
                yref='paper', 
                showarrow=False, 
                xanchor='left', 
                text='kde = 1, x = -9.6', # ex: sell: 10 USD price: 0.99995 // {i.type}: {i.amount} {i.main/sec currency label} {i.price}
                bgcolor='green',
                )],
    shapes = [dict(
    x0=-9.6, 
    x1=-9.6, 
    #pos of vertical line 
    y0=0,
    y1=1, 
    xref='x', 
    yref='paper', 
    line_width=1, 
    ) ]
)
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity
# calc later
fig

In [ ]:
assert 0
# ax.hist(flights['arr_delay'], bins = int(180/binwidth),
#              color = 'blue', edgecolor = 'black')

AssertionError: 

In [ ]:
# poggers   
server_time = datetime.fromtimestamp(OpenBinanceApi.server_time()/1000)
my_time = datetime.now()
delay = server_time - my_time                 
while 1:
    # pepe
    logging.info('get new')

    # server_time_now = (datetime.now() + delay)
    logging.info(f'server time: {datetime.fromtimestamp(OpenBinanceApi.server_time()/1000)}')
    df = OpenBinanceApi.get_df(
            pair = 'RVNUSDT',
            interval = '1m',
            limit = 1,
        ) 
    dt = datetime\
            .fromtimestamp(
                int(df.Real_Date[-1:])/1000
            )
    estimated_time = 0
    server_time = dt
    server_delay = timedelta(seconds=-1) # nedd to calc that in init
    dt_to_wait = server_time + delay + timedelta(seconds=60) + server_delay
# короче надо напистаь проверку которая чекает пришли данные свежие или нет
# если данные не свежие то надо :
# пересчитать серв делэй
# запросить данные занового пока не придут свежие
    logging.info(f'real time: {dt}')
    logging.info(f'dt: {dt_to_wait}')
    # logging.info(f'time to wait: {time_to_wait} s')
    # if time_to_wait > 0:
    # (real time min+1) > server time
        # await asyncio.sleep(1)
    await sleep_until(dt_to_wait)

    # else: 
    #     print('mission failed')
    #     break


2022-02-21 07:33:36,866 [INFO]: get new
2022-02-21 07:33:37,264 [INFO]: server time: 2022-02-21 07:33:42.334000
2022-02-21 07:33:37,645 [INFO]: real time: 2022-02-21 07:33:00
2022-02-21 07:33:37,646 [INFO]: dt: 2022-02-21 07:34:04.068603
2022-02-21 07:34:04,073 [INFO]: get new
2022-02-21 07:34:04,450 [INFO]: server time: 2022-02-21 07:34:09.520000
2022-02-21 07:34:05,340 [INFO]: real time: 2022-02-21 07:34:00
2022-02-21 07:34:05,342 [INFO]: dt: 2022-02-21 07:35:04.068603


In [ ]:
assert 0

AssertionError: 

In [ ]:
server_time = datetime.fromtimestamp(OpenBinanceApi.server_time()/1000)
my_time = datetime.now()
delay = server_time - my_time
display(f'{server_time=}')
display(f'{my_time=}')
delay.total_seconds()

'server_time=datetime.datetime(2022, 2, 21, 14, 21, 41, 935000)'

'my_time=datetime.datetime(2022, 2, 21, 14, 21, 37, 89441)'

4.845559

In [ ]:
server_time = datetime.fromtimestamp(OpenBinanceApi.server_time()/1000)
display('pc time', datetime.now())
# display(server_time - delay)
logging.info(f"server_time: {server_time}")
# server_time -> pc_time 
real_time = server_time - delay

real_time


'pc time'

datetime.datetime(2022, 2, 20, 12, 59, 19, 616596)

2022-02-20 12:59:19,622 [INFO]: server_time: 2022-02-20 12:59:22.562000


datetime.datetime(2022, 2, 20, 12, 59, 19, 609975)

In [ ]:
trade_data